In [3]:
# Functions in python are first class citizens . i.e. they are objects.
# And can be assigned to variables, returned from other functions and can be used as function arguments
def square(num):
    return num * num


print(id(square))  # print the memory address of the 'address' function object

f = square  # assigning the square function object to another variable
print(f(5))  # invoking the square function object through a reference, in this case 'f'

5545906944
25


In [6]:
# A higher order function - a function that receives another function as an argument or returns another function
# map() is a higher order function
squared_itr = map(square, [1, 2, 3, 4])
print(list(squared_itr))

[1, 4, 9, 16]


In [7]:
# Implementing our own higher order function
def my_map(func, iterable):
    result = []
    for elem in iterable:
        result.append(func(elem))
    return result


print(my_map(square, [1, 2, 3, 4]))

[1, 4, 9, 16]


In [8]:
# Another example of a higher order function - Return a function from another function
def logger(msg):
    def log_message():
        print("Log:", msg)

    return log_message


hi_log = logger('Hi')
hi_log()

Log: Hi


In [10]:
# Another example of a higher order function
def html_tag(tag):
    def wrap_text(msg):
        print(f"<{tag}> {msg} </{tag}>")

    return wrap_text


print_h1 = html_tag("h1")
print_h2 = html_tag("h2")

print_h1("Heading 1")  # NOTE: wrap_text() remembered the tag passed to html_tag. This is closure in action!
print_h2("Heading 2")

<h1> Heading 1 </h1>
<h2> Heading 2 </h2>


In [15]:
# CLOSURES:
# A closure is a function … but with “memory”
# So you can think of a closure as: a function bundled together with the variables it needs from its surrounding environment.
# When the inner function captures/remembers variables from outside itself, we say it "closes over" those variables.
# It's like the function wraps itself around those variables and holds onto them.

def outer_func(msg):
    def inner_func():
        print(msg)

    return inner_func


# Create a closure
# hi_func refers to inner_func function which is bundled with the remembered variable 'msg' from outer_func's scope
hi_func = outer_func("Hi")
print(hi_func.__name__)
hi_func()  # call the closure

inner_func
Hi


In [14]:
# When you do something like below code, outer_func returns the function object inner_func.
# That function object (inner_func) has an attribute called __closure__.
# __closure__ is a tuple of “cell” objects
hello_func = outer_func("Hello")
print(hello_func.__closure__)

# each cell storing one variable captured from the outer scope
print(hello_func.__closure__[0].cell_contents)


(<cell at 0x1573bedd0: str object at 0x14a8bdbb0>,)
Hello


In [1]:
# Practice
def outer_function(msg):
    def inner_function():
        print(msg)

    return inner_function


hi_function = outer_function("Hi")
hello_function = outer_function("Hello")

hi_function()
hello_function()

Hi
Hello


In [10]:
# A decorator in Python is a function that takes another function or class as input and
# extends or modifies its behavior without changing the original code of the function or class.

def decorator_function(original_function):
    def wrapper_function():
        return original_function()  # return the value(if any) returned from original function after executing it

    return wrapper_function


def display():
    print("Display function ran!!")


decorated_display = decorator_function(display)
decorated_display()

Display function ran!!


In [9]:
# so I can add more functionalities to my existing function by adding those functionalities to the wrapper
def decorator_function1(original_function):
    def wrapper_function():
        print(f"Wrapper executed this before {original_function.__name__}")
        return original_function()

    return wrapper_function


def display_msg():
    print("Saying hello from display_msg()")


display_msg = decorator_function1(display_msg)
display_msg()

Wrapper executed this before display_msg
Saying hello from display_msg()


In [11]:
# The above code can be written as
def decorator_function2(original_function):
    def wrapper_function():
        print(f"Wrapper executed this before {original_function.__name__}")
        return original_function()

    return wrapper_function


@decorator_function2
def display1():
    print("Inside display...")


display1()

# NOTE-the code:
# @decorator_function2
# def display():
#     print("Inside display...")
#
# is syntactical sugar for
# def display():
#     print("Inside display...")
#
# display = decorator_function2(display)
#
# So display now points to the wrapper function, waiting to be executed


Wrapper executed this before display1
Inside display...


In [14]:
# What is our original function took in any arguments? In such cases wrapper should take *args & **kwargs
# i.e. arbitrary number of positional and keyword arguments

def decorator_function3(original_function):
    def wrapper_function(*args, **kwargs):
        print(f"Executing wrapper before {original_function.__name__}")
        return original_function(*args, **kwargs)

    return wrapper_function


@decorator_function3
def display2():
    print("Inside display2")


@decorator_function3
def display_info(name, age):
    print(f"display_info() ran with arguments {name}, {age}")


display2()
display_info("Amitava", "2")


Executing wrapper before display2
Inside display2
Executing wrapper before display_info
display_info() ran with arguments Amitava, 2


In [15]:
# Decorators can be classes too
# We leverage the __call__() method for this
# Any object that implements the __call__() method becomes callable - meaning you can use parenthesis () to call it
# just like a function.
class Greeter:
    def __init__(self, message):
        self.message = message

    def __call__(self):
        print(f"Hello {self.message}")


greeter = Greeter("Amitava")
greeter()

Hello Amitava


In [17]:
class DecoratorClass:
    def __init__(self, original_function):
        self.original_function = original_function

    def __call__(self, *args, **kwargs):
        print(f"Calling Decorator class before {self.original_function.__name__}")
        return self.original_function(*args, **kwargs)


@DecoratorClass
def display3():
    print("Inside display3")


@DecoratorClass
def display_info2(name, age):
    print(f"Inside display_info2 with arguments {name}, {age}")


display3()
display_info2("Abc", "100")

# NOTE -
# When @DecoratorClass is applied, Python creates an instance of DecoratorClass,
# passing the function as an argument to its constructor (__init__).
# That instance (which is callable) replaces the original function name in the namespace.
# display3 = DecoratorClass(display3)
# display_info2 = DecoratorClass(display_info2)

Calling Decorator class before display3
Inside display3
Calling Decorator class before display_info2
Inside display_info2 with arguments Abc, 100


In [25]:
# Practical Example of decorators - timing how long a function ran


def timing_decorator(original_function):
    def timing_wrapper_function(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print(f"function {original_function.__name__} ran in {round(t2, 3)} seconds")
        return result

    return timing_wrapper_function


@timing_decorator
def sample_function():
    time.sleep(1)
    return "Done executing"


returned_value = sample_function()
print(returned_value)



function sample_function ran in 1.005 seconds
Done executing


In [26]:
# Another example of a decorator
import logging


def logging_decorator(original_function):
    logging.basicConfig(filename="sample.log", level=logging.INFO)

    def logging_wrapper_function(*args, **kwargs):
        logging.info(f"Executing function {original_function.__name__}")
        return original_function(*args, **kwargs)

    return logging_wrapper_function


@logging_decorator
def sample_function():
    print("Inside sample function...")


sample_function()



Inside sample function...


In [28]:
# If using two decorators
@logging_decorator
@timing_decorator
def another_sample_function():
    print("Inside another sample function..")


another_sample_function()

# Issue with using 2 decorators - The log would be - Executing function timing_wrapper_function
# Because :
# another_sample_function = logging_decorator(timing_decorator(another_sample_function))

Inside another sample function..
function another_sample_function ran in 0.0 seconds


In [37]:
import time
from functools import wraps


# How to solve this issue
def logging_decorator4(original_function):
    @wraps(original_function)
    def logging_wrapper(*args, **kwargs):
        logging.basicConfig(filename="sample.log", level=logging.INFO)
        logging.info(f"Executing function {original_function.__name__} with args {args} and keyword args {kwargs}")
        return original_function(*args, **kwargs)

    return logging_wrapper


def timing_decorator4(original_function):
    @wraps(original_function)
    def timing_wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print(f"Time taken to execute {original_function.__name__} is {round(t2, 3)} seconds.")
        return result

    return timing_wrapper


@logging_decorator4
@timing_decorator4
def display4_info(name, age):
    time.sleep(1)
    print(f"Inside display4_info. Args - {name}, {age}")


display4_info("Sam", "100")


Inside display4_info. Args - Sam, 100
Time taken to execute display4_info is 1.001 seconds.


In [38]:
# wraps() - Make the wrapper function look like the original function it’s decorating.
# @wraps copies important metadata (like name, docstring, annotations) from original function to wrapper.
def my_decorator(original_function):
    def my_wrapper():
        return original_function()

    return my_wrapper


@my_decorator
def say_hello():
    """say hello!"""
    print("hello")


print(say_hello.__name__)
print(say_hello.__doc__)

# Output:
# my_wrapper
# None
#
# That’s wrong — say_hello should still appear as say_hello, not wrapper. @wraps() fixes this.

my_wrapper
None


In [41]:
# functools.wraps() internally does: update_wrapper(wrapper, original_function)
# and that copies: __name__, __doc___, __module__ & other metadata
# So when debugging, profiling, or introspecting functions (like in help(), or logging), the correct metadata shows up.

def my_decorator_v2(original_function):
    @wraps(original_function)
    def my_wrapper(*args, **kwargs):
        return original_function(*args, **kwargs)

    return my_wrapper


@my_decorator_v2
def say_hello_v2():
    """say hello!"""
    print("say hello!")


print(say_hello_v2.__name__)
print(say_hello_v2.__doc__)

say_hello_v2
say hello!
